## Transfer Learning from InceptionV3

The idea of this step is to leverage existing state of the art models such as Ince

Borrowed some details from this example [https://www.kaggle.com/kmader/transfer-learning-with-inceptionv3](https://www.kaggle.com/kmader/transfer-learning-with-inceptionv3)

# Create image generators

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import json
from PIL import Image

In [3]:
def generate_generator_multiple(generator,directories, batch_size, img_height,img_width):
    generators =[]
    for directory in directories:
        gen = generator.flow_from_directory(directory,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
        generators.append(gen)

    for gen in generators:
        for data, labels in gen:
            yield data, labels

In [4]:
def save_plots(history,target_file_acc,target_file_loss):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(target_file_acc)
    plt.close()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(target_file_loss)
    plt.close()

In [ ]:
def show_plots(history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Doing K-Folds

In [ ]:
batch_size = 32
img_height=128
img_width = 256
approx_fold_size = 800

In [ ]:
urban_sounds_folder = "/home/nvidia/soundflux_data/spectrograms"

In [ ]:
fold_directories = []
for i in range(1,11):
    directory = urban_sounds_folder+"/fold"+str(i)
    fold_directories.append(directory)

In [ ]:
fold_directories

['/home/nvidia/soundflux_data/spectrograms/fold1',
 '/home/nvidia/soundflux_data/spectrograms/fold2',
 '/home/nvidia/soundflux_data/spectrograms/fold3',
 '/home/nvidia/soundflux_data/spectrograms/fold4',
 '/home/nvidia/soundflux_data/spectrograms/fold5',
 '/home/nvidia/soundflux_data/spectrograms/fold6',
 '/home/nvidia/soundflux_data/spectrograms/fold7',
 '/home/nvidia/soundflux_data/spectrograms/fold8',
 '/home/nvidia/soundflux_data/spectrograms/fold9',
 '/home/nvidia/soundflux_data/spectrograms/fold10']

# MODEL

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

In [1]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
from keras.applications.vgg16 import VGG16
from keras import metrics
input_shape = (img_height, img_width,3)
nclass = 10

base_model = VGG16(weights='imagenet', 
                                include_top=False, 
                                input_shape=(img_height, img_width,3))
base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
#add_model.add(Dense(2000,activation='relu'))
add_model.add(Dropout(0.5))
#add_model.add(Dense(2000,activation='relu'))
#add_model.add(Dense(2000,activation='relu'))
#add_model.add(Dense(512,activation='relu'))
add_model.add(Dense(nclass, 
                    activation='softmax'))

#set optimizer
#opt = optimizers.Adam(lr=0.05, beta_1=0.9, beta_2=0.999, decay=0.0)
opt = optimizers.RMSprop(lr=0.001)
model = add_model
model.compile(loss='categorical_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])
#needed to reset weights!
model.save_weights('raw_inception_model.h5')
model.summary()

Using TensorFlow backend.


NameError: name 'img_height' is not defined

# Folding

In [ ]:
num_folds = 10 #1 to 10
fold = 0
for directory in fold_directories:
    fold +=1
    #RESET WEIGHTS!!
    model.load_weights('raw_inception_model.h5')
    #
    train_directories = list(set(fold_directories) - set([directory]))
    test_directories = [directory]
    print("Running fold {}, holding data from {} and training on the remaining {}" \
          .format(fold,directory,len(train_directories)))
    train_generator = generate_generator_multiple(generator=datagen,
                                           directories = train_directories,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)
    test_generator = generate_generator_multiple(generator=datagen,
                                       directories = test_directories,
                                       batch_size=batch_size,
                                       img_height=img_height,
                                       img_width=img_width)
    history = model.fit_generator(train_generator,
                              steps_per_epoch=approx_fold_size*9/batch_size,
                              epochs=100,
                              validation_data = test_generator,
                              validation_steps=approx_fold_size/batch_size,
                              shuffle=True, 
                              verbose=True)
    model.save_weights('trained_inception_model_fold_{}.h5'.format(fold))
    with open('training_inception_history_fold_{}.json'.format(fold), 'w') as f:
        json.dump(history.history, f)
    save_plots(history,'training_inception_accuracy_plot_fold_{}.png'.format(fold),
               'training_inception_lossaccuracy_plot_fold_{}.png'.format(fold))
    if fold == num_folds:
        break

Running fold 1, holding data from /home/nvidia/soundflux_data/spectrograms/fold1 and training on the remaining 9
Epoch 1/100
Found 936 images belonging to 10 classes.
Found 873 images belonging to 10 classes.
Found 823 images belonging to 10 classes.
Found 990 images belonging to 10 classes.
Found 888 images belonging to 10 classes.
Found 925 images belonging to 10 classes.
Found 816 images belonging to 10 classes.
Found 837 images belonging to 10 classes.
Found 838 images belonging to 10 classes.
Found 806 images belonging to 10 classes.
225/225 [==============================] - 162s 719ms/step - loss: 2.6191 - acc: 0.2740 - val_loss: 4.8862 - val_acc: 0.1263
Epoch 2/100
225/225 [==============================] - 135s 598ms/step - loss: 2.4718 - acc: 0.3005 - val_loss: 6.3257 - val_acc: 0.0940
Epoch 3/100
225/225 [==============================] - 133s 593ms/step - loss: 2.4445 - acc: 0.2987 - val_loss: 5.0303 - val_acc: 0.1287
Epoch 4/100
225/225 [==============================] - 1

In [ ]:
show_plots(history)